In [ ]:
# =====================================================
# Imports
# =====================================================

import re
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px


In [ ]:
# =====================================================
# Data sets
# =====================================================

path_emissies = "Emissies_Nederland__wegverkeer_13102025_110220.csv"
path_km       = "Verkeer_motorvoertuigen_13102025_105929.csv"
path_electric = "Procentuele aandeel BEV_ FCEV en PHEV personenauto's in het wagenpark.csv"
path_EVshare = "Procentuele aandeel BEV_ FCEV en PHEV personenauto's in het wagenpark.csv"

In [8]:
# =====================================================
# Sub-question 1: How do CO₂, NOx, and PM10 totals and intensities (g/km) for passenger cars evolve over 2018–2023?  
# =====================================================

# Finds the first column name in the DataFrame that starts with the given string
def find_col(df, startswith):
    for c in df.columns:
        if str(c).startswith(startswith):
            return c
    return None 

# Read the emissions and kilometers CSVs
df_e = pd.read_csv(path_emissies, sep=';', encoding='utf-8-sig', quotechar='"', decimal=',')
df_km = pd.read_csv(path_km,       sep=';', encoding='utf-8-sig', quotechar='"', decimal=',')


# Identifies the relevant emission columns
co2_col  = find_col(df_e, "Emissies/Kooldioxide (CO2)")
nox_col  = find_col(df_e, "Emissies/Stikstofoxiden (NOx)")
pm10_col = find_col(df_e, "Emissies/PM10 Totaal")

# Keeps only passenger cars ("Personenauto's") and extracts the year from the 'Perioden' column
d_e = df_e[df_e['Voertuigtype'] == "Personenauto's"].copy()
d_e['Jaar'] = d_e['Perioden'].astype(str).str.extract(r'(\d{4})').astype(int)

# Limit to the years 2018–2023
d_e = d_e[(d_e['Jaar'] >= 2018) & (d_e['Jaar'] <= 2023)]


d_e = d_e[['Jaar', co2_col, nox_col, pm10_col]].rename(columns={
    co2_col:  'CO2_mlnkg',
    nox_col:  'NOx_mlnkg',
    pm10_col: 'PM10_mlnkg'
}).sort_values('Jaar')

km_total_col = find_col(df_km, "Kilometers in Nederland/Totaal kilometers in Nederland")

# Keeps only passenger cars and extracts year
d_km = df_km[df_km['Voertuigtypes'] == 'Personenauto'].copy()
d_km['Jaar'] = d_km['Perioden'].astype(str).str.extract(r'(\d{4})').astype(int)

# Keeps year and total kilometers, renames for clarity
d_km = d_km[['Jaar', km_total_col]].rename(columns={km_total_col: 'km_mln'}).sort_values('Jaar')

# Merges emissions and kilometers on year
df = d_e.merge(d_km, on='Jaar', how='inner')

# Calculates emission intensities in grams per kilometer
df['CO2_g_per_km']  = (df['CO2_mlnkg']  * 1000) / df['km_mln']
df['NOx_g_per_km']  = (df['NOx_mlnkg']  * 1000) / df['km_mln']
df['PM10_g_per_km'] = (df['PM10_mlnkg'] * 1000) / df['km_mln']

# Displays summary table
print("\nTable: kilometers (mln km), totals (mln kg), and intensities (g/km)")
print(df[['Jaar', 'km_mln', 'CO2_mlnkg', 'NOx_mlnkg', 'PM10_mlnkg',
          'CO2_g_per_km', 'NOx_g_per_km', 'PM10_g_per_km']].to_string(index=False))

# Generates three side-by-side line plots for given y-variables
def three_line_subplots(dataframe, x_col, y_cols, titles, ylabels, main_title):
    x_vals = dataframe[x_col].astype(str)
    fig = make_subplots(rows=1, cols=3, subplot_titles=titles, horizontal_spacing=0.08)
    for i, (y, ylabel) in enumerate(zip(y_cols, ylabels), start=1):
        fig.add_trace(
            go.Scatter(
                x=x_vals, y=dataframe[y],
                mode="lines+markers",
                marker=dict(size=7),
                line=dict(width=3),
                hovertemplate="Jaar: %{x}<br>" + f"{ylabel}: " + "%{y:.3f}<extra></extra>"
            ),
            row=1, col=i
        )
        fig.update_xaxes(title_text="Year", row=1, col=i)
        fig.update_yaxes(title_text=ylabel, rangemode="tozero", row=1, col=i)
    fig.update_layout(template="plotly_white", showlegend=False,
                      height=380, width=1100, title_text=main_title)
    return fig


# Plot total emissions
fig_totals = three_line_subplots(
    df, 'Jaar',
    y_cols=['CO2_mlnkg', 'NOx_mlnkg', 'PM10_mlnkg'],
    titles=['CO₂ total (mln kg)', 'NOx total (mln kg)', 'PM10 total (mln kg)'],
    ylabels=['CO₂ (mln kg)', 'NOx (mln kg)', 'PM10 (mln kg)'],
    main_title='Passenger cars — total emissions per year'
)
fig_totals.show()


# Plot emission intensities
fig_intensity = three_line_subplots(
    df, 'Jaar',
    y_cols=['CO2_g_per_km', 'NOx_g_per_km', 'PM10_g_per_km'],
    titles=['CO₂ intensity (g/km)', 'NOx intensity (g/km)', 'PM10 intensity (g/km)'],
    ylabels=['CO₂ (g/km)', 'NOx (g/km)', 'PM10 (g/km)'],
    main_title='Passenger cars — emission intensities (g/km)'
)
fig_intensity.show()





Table: kilometers (mln km), totals (mln kg), and intensities (g/km)
 Jaar   km_mln  CO2_mlnkg  NOx_mlnkg  PM10_mlnkg  CO2_g_per_km  NOx_g_per_km  PM10_g_per_km
 2018 109337.4      17487      28.79        2.25    159.936124      0.263313       0.020579
 2019 109086.8      17230      26.73        2.19    157.947616      0.245034       0.020076
 2020  91723.0      14302      19.58        1.81    155.925995      0.213469       0.019733
 2021  96988.8      14874      19.15        1.90    153.357913      0.197445       0.019590
 2022 102679.6      15374      18.90        2.00    149.727891      0.184068       0.019478
 2023 107143.1      15638      18.66        2.08    145.954336      0.174160       0.019413


In [11]:
# =====================================================
# Sub-question 2: How does the BEV, FCEV, PHEV (passenger-car) population develop during the period 2018-2023? 
# =====================================================

# Used to parse dates written in Dutch like "31 december 2023"
MONTHS_NL = {
    "januari":1, "februari":2, "maart":3, "april":4, "mei":5, "juni":6,
    "juli":7, "augustus":8, "september":9, "oktober":10, "november":11, "december":12
}

# Accepts formats like "15 maart 2021" and returns a pandas Timestamp
def parse_dutch_date(s: str):
    s = str(s).strip().lower()
    m = re.match(r"(\d{1,2})\s+([a-z]+)\s+(\d{4})", s)
    if not m:
        return pd.NaT
    day = int(m.group(1)); month = MONTHS_NL.get(m.group(2)); year = int(m.group(3))
    return pd.Timestamp(year=year, month=month, day=day) if month else pd.NaT

# Percentage cleaner (keeps % if present; handles 0–1 or 0–100 inputs)
def to_pct_keep(x):
    raw = str(x)
    s = raw.strip().replace(",", ".").replace("%", "")
    try:
        val = float(s)
    except Exception:
        return np.nan
    return val if "%" in raw else (val * 100.0 if val <= 1.0 else val)

# Loads file and keeps last observation per year
def load_last_per_year(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename, sep=";", engine="python", encoding="utf-8-sig")

    # Fixes headers: takes labels from first row, keeps the first column name as 'Datum'
    new_cols = list(df.columns)
    header = df.iloc[0].tolist()
    new_cols[0] = "Datum"
    for i in range(1, len(new_cols)):
        lab = header[i]
        if isinstance(lab, str) and lab.strip():
            new_cols[i] = lab.strip()
    df.columns = new_cols
    df = df.iloc[1:].reset_index(drop=True)

    # Parses date and derives year
    df["date"] = df["Datum"].apply(parse_dutch_date)
    df["jaar"] = df["date"].dt.year

    # Normalize percentage-like columns to percent scale
    for col in ["BEV", "FCEV", "PHEV"]:
        df[col] = df[col].apply(to_pct_keep)

    # Pick the last (max date) record per year and compute combined share
    idx_last = df.groupby("jaar")["date"].idxmax()
    last = (df.loc[idx_last, ["jaar", "BEV", "FCEV", "PHEV"]]
              .sort_values("jaar")
              .reset_index(drop=True))
    last["BEV+PHEV+FCEV"] = last[["BEV","PHEV","FCEV"]].sum(axis=1)
    return last[["jaar","BEV+PHEV+FCEV"]]

ts_last = load_last_per_year(path_EVshare)

# Keep years up to (and including) 2023
ts_last = ts_last[ts_last["jaar"] <= 2023].reset_index(drop=True)

# Plot: Combined share of BEV/PHEV/FCEV (percentage)
fig = px.line(
    ts_last,
    x="jaar",
    y="BEV+PHEV+FCEV",
    markers=True,
    title="Amount(%) of BEV + PHEV + FCEV on the road"
)

# Layout and interaction changes
fig.update_layout(
    xaxis_title="year",
    yaxis_title="Amount (%)",
    template="plotly_white",
    hovermode="x unified",
)
fig.update_yaxes(range=[0, 20])
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Aandeel: %{y:.2f}%<extra></extra>"
)

fig.show()


In [14]:
# =====================================================
# Sub-question 3: How are BEV, FCEV, PHEV (passenger-car) changes related to annual emission intensities?
# =====================================================

def find_col(df, startswith):
    for c in df.columns:
        if str(c).startswith(startswith):
            return c
    return None

# Loads raw emissions and kilometers datasets
df_e = pd.read_csv(path_emissies, sep=';', encoding='utf-8-sig', quotechar='"', decimal=',')
df_km = pd.read_csv(path_km, sep=';', encoding='utf-8-sig', quotechar='"', decimal=',')

# Identifies emissions columns for CO2, NOx, and PM10
co2_col  = find_col(df_e, "Emissies/Kooldioxide (CO2)")
nox_col  = find_col(df_e, "Emissies/Stikstofoxiden (NOx)")
pm10_col = find_col(df_e, "Emissies/PM10 Totaal")

# Filters emissions to passenger cars and years of interest
d_e = df_e[df_e['Voertuigtype'] == "Personenauto's"].copy()
d_e['Jaar'] = d_e['Perioden'].astype(str).str.extract(r'(\d{4})').astype(int)
d_e = d_e[(d_e['Jaar'] >= 2018) & (d_e['Jaar'] <= 2023)]
d_e = d_e[['Jaar', co2_col, nox_col, pm10_col]].rename(columns={
    co2_col:'CO2_mlnkg', nox_col:'NOx_mlnkg', pm10_col:'PM10_mlnkg'
}).sort_values('Jaar')

# Prepare kilometers (total km for passenger cars)
km_total_col = find_col(df_km, "Kilometers in Nederland/Totaal kilometers in Nederland")
d_km = df_km[df_km['Voertuigtypes'] == 'Personenauto'].copy()
d_km['Jaar'] = d_km['Perioden'].astype(str).str.extract(r'(\d{4})').astype(int)
d_km = d_km[['Jaar', km_total_col]].rename(columns={km_total_col:'km_mln'}).sort_values('Jaar')

# Merges emissions with kilometers and computes intensities (g/km)
df_int = d_e.merge(d_km, on='Jaar', how='inner')
df_int['CO2_g_per_km']  = (df_int['CO2_mlnkg']  * 1000) / df_int['km_mln']
df_int['NOx_g_per_km']  = (df_int['NOx_mlnkg']  * 1000) / df_int['km_mln']
df_int['PM10_g_per_km'] = (df_int['PM10_mlnkg'] * 1000) / df_int['km_mln']


# Loads yearly mean EV shares (BEV/FCEV/PHEV) from files
def load_yearly_means(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename, sep=";", engine="python", encoding="utf-8-sig")

    # Fixes headers using first row, keeps first column name as 'Datum'
    new_cols = list(df.columns)
    first_row = df.iloc[0].tolist()
    new_cols[0] = "Datum"
    for i in range(1, len(new_cols)):
        lab = first_row[i]
        if isinstance(lab, str) and lab.strip():
            new_cols[i] = lab.strip()
    df.columns = new_cols
    df = df.iloc[1:].reset_index(drop=True)

    # Extracts year from 'Datum'
    def parse_year(s): 
        m = re.search(r"(\d{4})", str(s))
        return int(m.group(1)) if m else np.nan
    df["Jaar"] = df["Datum"].apply(parse_year).astype("Int64")
    df = df[df["Jaar"].notna()].copy()
    df["Jaar"] = df["Jaar"].astype(int)

    # Normalizes percentage-like cells
    def to_pct_keep(x):
        raw = str(x); s = raw.strip().replace(",", ".").replace("%", "")
        try: val = float(s)
        except: return np.nan
        if "%" in raw: return val
        return val * 100 if val <= 1 else val

    # Cleans BEV/FCEV/PHEV and compute yearly mean shares
    for col in ["BEV", "FCEV", "PHEV"]:
        if col in df.columns: df[col] = df[col].apply(to_pct_keep)
    ts = df.groupby("Jaar", as_index=False)[["BEV", "FCEV", "PHEV"]].mean(numeric_only=True)
    return ts.sort_values("Jaar")

# Loads EV shares and computes total EV share
df_ev = load_yearly_means(path_electric)
df_ev["Total_EV_share"] = df_ev[["BEV", "FCEV", "PHEV"]].sum(axis=1)

# Combines EV shares with emission intensities
combo = df_int.merge(df_ev, on="Jaar", how="inner")

# print combined dataset
print("\nCombined dataset (aandeel EV en emissie-intensiteiten):")
print(combo[["Jaar", "BEV", "PHEV", "FCEV", "Total_EV_share",
             "CO2_g_per_km", "NOx_g_per_km", "PM10_g_per_km"]].to_string(index=False))

# dual-axis chart for EV share vs. a chosen pollutant intensity
def dual_plot(y1, y2, label1, label2, title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Left axis: EV share (%)
    fig.add_trace(go.Scatter(x=combo["Jaar"], y=combo[y1],
                             mode="lines+markers", name=label1,
                             line=dict(width=3, color="#1f77b4")), secondary_y=False)
    # Right axis: pollutant intensity (g/km)
    fig.add_trace(go.Scatter(x=combo["Jaar"], y=combo[y2],
                             mode="lines+markers", name=label2,
                             line=dict(width=3, dash="dash", color="#ff7f0e")),
                  secondary_y=True)
    fig.update_xaxes(title_text="Year")
    fig.update_yaxes(title_text=f"{label1} (%)", secondary_y=False)
    fig.update_yaxes(title_text=f"{label2} (g/km)", secondary_y=True)
    fig.update_layout(title_text=title, template="plotly_white", width=900, height=450)
    fig.show()

# Visualize relations: EV share vs. each pollutant intensity
dual_plot("Total_EV_share", "CO2_g_per_km", "EV share", "CO₂ intensity", 
          "Relation between EV share and CO₂ intensity (2018–2023)")
dual_plot("Total_EV_share", "NOx_g_per_km", "EV share", "NOx intensity",
          "Relation between EV share and NOx intensity (2018–2023)")
dual_plot("Total_EV_share", "PM10_g_per_km", "EV share", "PM10 intensity",
          "Relation between EV share and PM10 intensity (2018–2023)")

# Correlation diagnostics: simple Pearson r between EV share and intensities
corrs = {}
for pol, col in [("CO₂", "CO2_g_per_km"), ("NOx", "NOx_g_per_km"), ("PM10", "PM10_g_per_km")]:
    corrs[pol] = combo["Total_EV_share"].corr(combo[col])
print("\nCorrelations between EV share and emission intensities:")
for k, v in corrs.items():
    print(f"{k}: r = {v:.3f}")



Combined dataset (aandeel EV en emissie-intensiteiten):
 Jaar   BEV  PHEV  FCEV  Total_EV_share  CO2_g_per_km  NOx_g_per_km  PM10_g_per_km
 2018 0.375 1.150   0.0           1.525    159.936124      0.263313       0.020579
 2019 0.850 1.100   0.0           1.950    157.947616      0.245034       0.020076
 2020 1.575 1.125   0.0           2.700    155.925995      0.213469       0.019733
 2021 2.350 1.425   0.0           3.775    153.357913      0.197445       0.019590
 2022 3.325 1.925   0.0           5.250    149.727891      0.184068       0.019478
 2023 4.425 2.600   0.0           7.025    145.954336      0.174160       0.019413



Correlations between EV share and emission intensities:
CO₂: r = -0.996
NOx: r = -0.922
PM10: r = -0.836


In [17]:
# =====================================================
# Sub-question 4: How might the long-term growth in EV share affect emission intensity by 2030?
# =====================================================

# Extracts observed series from combined dataset
years_obs   = combo["Jaar"].values
ev_obs      = combo["Total_EV_share"].values       
co2_obs     = combo["CO2_g_per_km"].values           

# Linear trend in EV share over time
m_ev, b_ev  = np.polyfit(years_obs, ev_obs, 1)

# Builds a year axis for the forecast display (ends at 2030)
years_fut   = np.arange(years_obs.max()-5, 2031)

# Predicted EV share from the time-trend model
ev_pred     = m_ev * years_fut + b_ev

# Links CO₂ intensity to EV share linearly
m_co2, b_co2 = np.polyfit(ev_obs, co2_obs, 1)

# Projects CO₂ intensity by plugging the predicted EV share into the CO₂~EV model
co2_pred     = m_co2 * ev_pred + b_co2

# Visualization: dual-axis time chart of observed & projected series
fig = go.Figure()

# Observed CO₂ intensity
fig.add_trace(go.Scatter(
    x=years_obs, y=co2_obs,
    mode="lines+markers",
    name="Observed CO₂ intensity (2018–2023)",
    line=dict(width=3, color="#3b6cff"),
    marker=dict(size=8),
    hovertemplate="Year %{x}<br>CO₂: %{y:.1f} g/km<extra></extra>"
))

# Projected CO₂ intensity
fig.add_trace(go.Scatter(
    x=years_fut, y=co2_pred,
    mode="lines",
    name="Projected CO₂ intensity (2023–2030)",
    line=dict(width=3, color="red", dash="dash"),
    hovertemplate="Year %{x}<br>CO₂ (proj): %{y:.1f} g/km<extra></extra>"
))

# Observed EV share on secondary y-axis
fig.add_trace(go.Scatter(
    x=years_obs, y=ev_obs,
    mode="lines+markers",
    name="Observed EV share (%)",
    line=dict(width=2, color="#2ca02c"),
    marker=dict(size=7),
    yaxis="y2",
    hovertemplate="Year %{x}<br>EV share: %{y:.1f}%<extra></extra>"
))

# Predicted EV share on secondary y-axis
fig.add_trace(go.Scatter(
    x=years_fut, y=ev_pred,
    mode="lines",
    name="Predicted EV share (%)",
    line=dict(width=2, color="#2ca02c", dash="dot"),
    yaxis="y2",
    hovertemplate="Year %{x}<br>EV share (proj): %{y:.1f}%<extra></extra>"
))

# Layout
fig.update_layout(
    title="Projected impact of EV share on CO₂ emission intensity (to 2030)",
    template="plotly_white",
    width=950, height=520,
    xaxis=dict(title="Year", dtick=1),
    yaxis=dict(title="CO₂ intensity (g/km)", rangemode="tozero"),
    yaxis2=dict(title="EV share (%)", overlaying="y", side="right", rangemode="tozero"),
    legend=dict(
        x=1.05, y=0.9,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    margin=dict(r=180)
)

fig.show()
